In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Natural Language Processing").getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover,CountVectorizer, IDF,StringIndexer, VectorAssembler
from pyspark.sql import functions as f
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

# Load Data

In [3]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

#### Dataset is taken from UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



# Data Processing

In [7]:
data = data.withColumn('length',f.length(data.text))

In [8]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### Feature Transformer

In [10]:
tokenizer = Tokenizer(inputCol="text", outputCol="tokenText")

In [11]:
stopWordsRemover = StopWordsRemover(inputCol="tokenText", outputCol="stopToken")

In [12]:
hamSpamToNumeric = StringIndexer(inputCol="class", outputCol="label")

### Feature Extraction

In [13]:
countVectorizer = CountVectorizer(inputCol="stopToken", outputCol="CountVector")

In [14]:
idf = IDF(inputCol="CountVector", outputCol="tfIDF")

### Train and Test Data Preparation

In [15]:
assembler = VectorAssembler(inputCols=["tfIDF","length"], outputCol="features")

In [16]:
dataPipeline = Pipeline(stages =[hamSpamToNumeric,tokenizer,stopWordsRemover,countVectorizer,idf,assembler])

In [17]:
cleanData = dataPipeline.fit(data).transform(data)
cleanData.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|           tokenText|           stopToken|         CountVector|               tfIDF|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [18]:
cleanData = cleanData.select('label','features')
cleanData.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [19]:
(train_data, test_data) = cleanData.randomSplit([0.7,0.3])

# Build Model

In [20]:
classifier = NaiveBayes()

In [21]:
model = classifier.fit(train_data)

# Evaluate Model

In [22]:
predictions = model.transform(test_data)

In [23]:
predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-794.36008605233...|[1.0,1.1862085150...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1193.4656154564...|[1.0,1.1955934999...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-535.65194331838...|[1.0,9.6189256680...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-865.71195035301...|[1.0,3.5335580476...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-685.39356522057...|[1.0,8.7224552127...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-965.36221961011...|[1.0,6.5710546129...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-754.22625045170...|[1.0,1.7153159053...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-356.66100768461...|[1.0,4.7333543233...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1522.6841291557...|[0.99866395194657...|       0.0|
|  0.0|(13424,[0

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(predictions)
print("Accuracy of model at predicting spam was: {}".format(accuracy))

Accuracy of model at predicting spam was: 0.9144295363023566
